Lior Bassel - ID - 033536798
Eliezer Hadad - ID - 022766364

In [31]:
import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd

resultsNew = list()
Image_num = {}

################### Auxiliary functions  ################################

def clean_number_string(number_string):
    # Remove non-numeric characters except periods and commas
    cleaned_string = re.sub(r'[^\d.,]', '', number_string)
    # Replace commas with empty strings
    cleaned_string = cleaned_string.replace(',', '')
    return cleaned_string
    
def string_to_float(s):
    if s.strip():  # Check if the string is not empty after stripping whitespace
        try:
            return float(s)
        except ValueError:
#           print(f"Error: Could not convert '{s}' to float.")
            return None

def days_until_current_date(target_date_str):
    # Parse the target date string into a datetime object
    target_date = datetime.strptime(target_date_str, '%m/%Y')
    
    # Get the current date
    current_date = datetime.today()
    
    # Calculate the difference in days
    difference = target_date - current_date 
    
    # Extract the number of days from the difference
    days_until_current = difference.days
    
    return days_until_current

########################################################################

# web page for Hyundai between years 1980-2013
response = requests.get("https://www.ad.co.il/car?sp261=13895&rp264=1980,2013")

generalUrl = BeautifulSoup(response.content,'html.parser')

# find Pic_Num
divs = generalUrl.find_all('div' , class_= 'card-block') 
for div in divs:
    if div:
       Pic_Num = div.get('data-images')
       adNumber = div.get('data-id')
       if adNumber is not None:
          key = "Pic_Num"
          value = Pic_Num
          Image_num[adNumber] = {'key': key, 'Pic_Num' : Pic_Num, 'value': value}

ads = generalUrl.find_all('div', class_= 'card-body p-md-3')

for ad in ads:
    # adId = ad.find('a').get_text())
    test= ad.find('a',recursive = False)
    if test == None:
        break
    newname = test.get('href', None)
    adId = (re.search(r'(?<=ad/)\d+', newname)).group()
#   print(adId)
    car_dict = {}
    response = requests.get("https://www.ad.co.il/ad/" + adId)
    soup = BeautifulSoup(response.content,'html.parser')
    table = soup.find('table',{'class' : 'table table-sm mb-4'})
    if table: 
              
              dataimages = soup.find('div',{'class':'card-block'}) if 'div' else None
              Pic_num = dataimages.get('data-images')
#             Pic_num= dataimages.find('div', class_='data-images') if 'div' else None
              dates = soup.find_all('div', class_='px-3')
              titles = soup.find_all('h2', class_='card-title')
             # Separate the contents
              if len(titles) > 0:
                  Cre_dateFull = dates[0].get_text() if dates[0] else None
                  Cre_date = Cre_dateFull.split(': ')[-1]
                  key = "Cre_date"
                  value = Cre_date
                  car_dict[key] = value
                  Repub_date = dates[1].get_text() if len(dates) > 1 else None
                  Repub_date = Repub_date.split(': ')[-1]
                  key = "Repub_date"
                  value = Repub_date
                  car_dict[key] = value
              else:
                  sourceDate = None
                  jumpingDate = None
              titles = soup.find_all('h2', class_='card-title')
             # Separate the contents
              if len(titles) > 0:
                  title_1 = titles[0].get_text() if titles[0] else None
                  title_2 = titles[1].get_text() if len(titles) > 1 else None
              else:
                  title_1 = None
                  title_2 = None
              car_string =  title_1.split()
              Manufacturer = car_string[0].strip()
              model = car_string[1].strip()
              key = "Manufacturer"
              value = Manufacturer
              car_dict[key] = value
              key = "model"
              value = model
              car_dict[key] = value
              key = "Price"
              value = title_2
              # Define a regex pattern to match digits and optionally a comma for thousands separator
              pattern = r'(\d{1,3}(?:,\d{3})*|\d+)'
              # Use regex to find the price
              matches = re.findall(pattern, value)
              # Extract the first match (which should be the price)
              if matches:
                  value = matches[0]
              else:
                  pass
              value = clean_number_string(value)
              string_to_float(value)
              car_dict[key] = value 
              Description = soup.find('p',{'class':'text-word-break'}) if 'p' else None
              Description = Description.get_text().strip() if Description else None
              key = "Description"
              value = Description
              car_dict[key] = value
              # print("Table found")
              trTable= table.find_all("tr")
              for tr in trTable:
                  td = tr.find_all('td')
                  if td !='':
                      key = td[0].get_text().strip()
                      value = td[1].get_text().strip()
                      if key == 'טסט עד' and value != None :
                         value = days_until_current_date(value)
                      car_dict[key] = value  # Store the key-value pair in the dictionary
              if adId in Image_num:
                  car_dict['Pic_Num'] = Image_num[adId]['Pic_Num']
              resultsNew.append(car_dict)  
    else:
       print("Table not found")



# Print the car details dictionary
df = pd.DataFrame(resultsNew)

# Define a dictionary to map old column names to new column names
new_column_names = {
    "יד": "Hand",
    "שנה": "Year",
    "ת. הילוכים": "Gear",
    "נפח": "Engine_capacity",
    "ק\"מ": "Km",
    "טסט עד": "Test",
    "צבע": "Color",
    "בעלות קודמת": "Prev_ownership",
    "בעלות נוכחית": "Curr_ownership",
    "אזור": "Area",
    "עיר": "City",
    "סוג מנוע": "Engine_type"
}

# Rename the columns
df.rename(columns=new_column_names, inplace=True)

# Function to create DataFrame and convert data types
def create_dataframe(df):
  if not df.empty:
        df = pd.DataFrame(df)
        df['Year'] = df['Year'].astype(int)  # Convert to integer
        df['Hand'] = df['Hand'].astype(int)  # Convert to integer
        df['Gear'] = df['Gear'].astype('category')  # Convert to categorical
        df['Engine_capacity'] = pd.to_numeric(df['Engine_capacity'], errors='coerce', downcast='integer')
        df['Prev_ownership'] = df['Prev_ownership'].astype('category')  # Convert to categorical
        df['Curr_ownership'] = df['Curr_ownership'].astype('category')  # Convert to categorical
        df['Area'] = df['Area'].astype(str)  # Convert to string
        df['City'] = df['City'].astype(str)  # Convert to string
        df['Price'] = pd.to_numeric(df['Price'], errors='coerce') # Convert to float
        df['Pic_Num'] = df['Pic_Num'].astype(int)  # Convert to integer
        df['Repub_date'] = pd.to_datetime(df['Repub_date'], errors='coerce')
        df['Description'] = df['Description'].astype(str)  # Convert to string
        df['Color'] = df['Color'].astype(str)  # Convert to string
        df['Km'] = pd.to_numeric(df['Km'], errors='coerce', downcast='integer')
        df['Test'] = pd.to_numeric(df['Test'], errors='coerce', downcast='integer')
        return df
  else:
        return None

create_dataframe(df)

# Export to CSV
csv_file_path = 'car_data.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')  # Specify UTF-8 encoding